<h2 style="font-weigt:bold;">TUGAS BESAR B MACHINE LEARNING</h2>

- Arjuna Marcelino - 13519021
- Sharon Bernadetha Marbun - 13519092
- Epata Tuah - 13519120
- Giant Andreas Tambunan - 13519127

### Membaca Data dan Model

In [72]:
import pandas as pd
import numpy as np
import math


# Baca Dataset
def readData():
    df = pd.read_csv('iris.csv')
    df['species'] = df['species'].replace(['setosa'],1)
    df['species'] = df['species'].replace(['versicolor'],2)
    df['species'] = df['species'].replace(['virginica'],3)
    y = pd.get_dummies(df.species, prefix='Class')
    df["species_setosa"] = y["Class_1"]
    df["species_versicolor"] = y["Class_2"]
    df["species_virginica"] = y["Class_3"]
    return df

### Fungsi Aktivasi

In [73]:
def linear(x, kwargs=None):
    return x

def sigmoid(x):
    value = float(1 / (1 + math.exp(x * -1)))
    return value
    '''threshold = 0.1
    if value < threshold:
        return 0
    else:
        return 1'''

def relu(x):
    alpha = 0.0
    max_value = 1.0
    threshold = 0.0
    if x < threshold:
        if x > x*alpha:
            return x
        else:
            return x*alpha
    else:
        return min(x, max_value)

def softmax(arr):
    arr_exp = np.exp(arr)
    return arr_exp / arr_exp.sum()

# Fungsi Loss Sum Squared Error untuk Linear, Sigmoid, and Relu activation
def lossSumSquareddError(x, t, o, outputs): 
    sumE = 0
    for i in outputs:
        sumE += (t[i] - o[i])**2
    return 1/2 * sumE

# Fungsi Loss untuk softmax
def lossCrossEntropy(pk):
    return -1*math.log(pk)

# Fungsi Turunan Relu
def turunanRelu(x):
    return 0 if x < 0 else 1
    
# Fungsi Turunan Sigmoid
def turunanSigmoid(x):
    return sigmoidActivation(x) * (1-sigmoidActivation(x))

# Fungsi Turunan Softmax
def turunanSoftmax(x, targetClass):
    return (-(1-x)) if targetClass else x

### Variabel Global

In [74]:
### Variabel Global

In [75]:
learningRate = 0.05
errorThreshold = 0.05
maxIteration = 100
batchSize = 2

In [76]:
# FORWARD PROPAGATION
def forwardPropagation(inputLayer, layer, activation, numNeuron):
    outputLayer = []
    for i in range (numNeuron):
        net = 0
        for j in range (len(inputLayer)+1):
            if (j==0):
                net += layer[0].weight[j][i] * layer[0].bias[i]
            else:
                net += layer[0].weight[j][i] * inputLayer[j-1]
        if activation == "linear":
            outputLayer.append(linear(net))
        elif activation == "sigmoid" :
            outputLayer.append(sigmoid(net))
        elif activation == "relu":
            outputLayer.append(relu(net))
        elif activation == "softmax" :
            outputLayer.append(softmax(net))
    return outputLayer

# BACKWARD PROPAGATION
def backwardPropagation(learningRate, preLayer, preLayerValue, outputs, currLayer, isOutput, targets, errorOValue):
    errorO = []
    errorH = []
    for i in range (len(currLayer)):
        if isOutput:
            errorO.append((targets[i] - outputs[i])*outputs[i]*(1-outputs[i]))
            # update weight
            for j in range (len(preLayerValue)):
                deltaW = learningRate * errorO[i] * preLayerValue[j]
            for k in range (len(preLayer[0].weight)):
                preLayer[0].weight[k][0] += deltaW
        else: # not output
            for j in range (len(currLayer)):
                sumWeightError = []
                for k in range (1, len(currLayer[j].weight)):
                    sumWeightError.append(currLayer[j].weight[k][j] * errorOValue[0])
            for j in range (len(preLayer)):
                for h in range (len(outputs)):
                    errorH.append(outputs[h]*(1-outputs[h]) * sumWeightError[h])
            # update weight
            deltaW = []
            insideDeltaW = []
            for j in range (len(preLayer)):
                for k in range (len(preLayerValue)):
                    insideDeltaW.append(learningRate * errorOValue[0] * -1)
                deltaW.append(insideDeltaW)
            for j in range (len(preLayer)):
                for k in range (len(preLayer[j].weight)):
                    for i in range (len(preLayer[j].weight[k])):
                        preLayer[j].weight[k][i] += deltaW[j][k-1]
                        #print("weight hasil: " + str(preLayer[j].weight[k][i]))
    return errorO if isOutput else errorH
    


### Kelas Layer

In [77]:
class Layer:
    def __init__(self, bias, weight, activation):
        self.weight = weight
        self.bias = bias
        self.activation = activation
    
    def getDiGraph(self, index, max):
        # untuk visualisasi Digraph
        arr = []

        # bias
        i = 1
        for bs in self.bias:
            a = f'b{index}'
            b = f'h{index+1}_{i}'
            if(index+1 == max):
                b = f'y'
            arr.append([a, b, bs])
            i += 1
        
        # weight 
        i = 1
        for x in self.weight:
            j = 1
            for w in x:
                a = f'x{index}_{i}'
                if(index>0):
                    a = f'h{index}_{i}'
                
                b = f'h{index+1}_{j}'
                if(index+1 == max):
                    b = f'y'
                arr.append([a, b, w])
                
                j += 1
            i += 1

        return arr

### Visualization


In [78]:
def createGraph(layerA, layerB, layerC):
    from graphviz import Digraph

    graph = Digraph("Backpropagation Model", filename="graph.gv")

    idx = 0
    max_idx = len(layerA) + len(layerB) + len(layerC)
    arr = []
    # LAYER A (INPUT)
    for i in range(len(layerA)):
        arr = layerA[i].getDiGraph(idx, max_idx)
        idx += 1
        for ed in arr:
            graph.edge(ed[0], ed[1], str(ed[2]))

    arr = []
    # LAYER B (HIDDEN)
    for i in range(len(layerB)):
        arr = layerB[i].getDiGraph(idx, max_idx-1)
        idx += 1
        for ed in arr:
            graph.edge(ed[0], ed[1], str(ed[2]))

    arr = []

    graph.view()

### Eksekusi

In [79]:
def predict(target, output):
        for i in range(len(output)):
            if(output[i] > 0.05):
                o = 1
            else:
                o = 0
            if(o != target):
                return False
        return True

def learn(data, maxIteration, errorThreshold, target):
    iteration = 0
    correct = 0
    while iteration < maxIteration:
        iteration += 1
        for index, item in data.iterrows():
            error = 0
            inputLayer = []
            hiddenLayer = []
            outputLayer = []
            inputLayer.append(Layer([1,1,1,1,1], [[-0.3,0.3,0.3,0.3],[0.2,0.3,-0.1,0.3],[0.3,0.1,-0.1,0.3],[0.3,0.1,-0.1,0.3],[-0.3,0.3,0.3,0.3]], 'sigmoid'))
            hiddenLayer.append(Layer([1], [[-0.1],[0.5],[-0.3],[-0.4],[0.1]], 'sigmoid'))
            outputLayer.append(Layer([1], [[-0.1],[0.5],[-0.3],[-0.4],[0.1]], 'sigmoid'))
            res1 = forwardPropagation([item['sepal_length'], item['sepal_width'], item['petal_length'], item['petal_width']], inputLayer, "sigmoid", 4)
            res2 = forwardPropagation(res1, hiddenLayer, "sigmoid", 1)
            if (item['species']==1):
                if predict(item['species_setosa'], res2):
                    correct += 1
            elif (item['species']==2):
                if predict(item['species_versicolor'], res2):
                    correct += 1
            elif (item['species']==3):
                if predict(item['species_virginica'], res2):
                    correct += 1

            if ((index+1) % batchSize == 0 or index == len(data)):
                errOutput = backwardPropagation(learningRate, hiddenLayer, res1, res2, outputLayer, True, [0], [])
                errFinal = backwardPropagation(learningRate, inputLayer, [1,1,1,1,1], res1, hiddenLayer, False, [], errOutput)
                for x in errFinal:
                    error += x
                if (error < errorThreshold):
                    print(inputLayer[0].weight)
                    createGraph(inputLayer, hiddenLayer, outputLayer)
                    return(inputLayer[0].weight), correct, iteration
            
                
data = readData()
result, correct, iteration = learn(data, maxIteration, errorThreshold, [0,1,2])

print("accurracy: " + str(correct/(iteration+1) * 100) + "%")




[[-0.2936983646790215, 0.3063016353209785, 0.3063016353209785, 0.3063016353209785], [0.2063016353209785, 0.3063016353209785, -0.09369836467902153, 0.3063016353209785], [0.3063016353209785, 0.10630163532097849, -0.09369836467902153, 0.3063016353209785], [0.3063016353209785, 0.10630163532097849, -0.09369836467902153, 0.3063016353209785], [-0.2936983646790215, 0.3063016353209785, 0.3063016353209785, 0.3063016353209785]]
accurracy: 100.0%
